In [1]:
import numpy as np
import pandas as pd
import emoji

from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, SimpleRNN,LSTM, Activation
from keras.utils import to_categorical
import matplotlib.pyplot as plt


In [2]:
train=pd.read_csv('D:\Data Sets\emoji_train.csv',header=None)
test=pd.read_csv('D:\Data Sets\emoji_test.csv',header=None)

In [3]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [4]:
emoji_dict = { 0 : ":heart:", 1 : ":baseball:", 2 : ":smile:", 3 : ":disappointed:", 4 : ":fork_and_knife:"}

In [5]:
for ix in emoji_dict.keys():
    print (ix,end=" ")
    print (emoji.emojize(emoji_dict[ix], use_aliases=True))

0 ❤
1 ⚾
2 😄
3 😞
4 🍴


In [6]:
X_train = train[0]
Y_train = train[1]

X_test = test[0]
Y_test = test[1]

print (X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(132,) (132,) (56,) (56,)


In [7]:
for i in range(5):
    print( X_train[i],emoji.emojize( emoji_dict[(Y_train[i])], use_aliases=True ))

never talk to me again 😞
I am proud of your achievements 😄
It is the worst day in my life 😞
Miss you so much ❤
food is life 🍴


In [8]:
f=open('D:\Data Sets\glove.6B.50d.txt',encoding='utf-8')

In [9]:
embeddings_index={}

for line in f:
    values=line.split()
    word=values[0]
    coef=np.asarray(values[1:],dtype='float')
    
    embeddings_index[word]=coef
f.close()

In [10]:
embeddings_index["eat"]

array([ 6.4295e-01, -4.2946e-01, -5.4277e-01, -1.0307e+00,  1.2056e+00,
       -2.7174e-01, -6.3561e-01, -1.5065e-02,  3.7856e-01,  4.6474e-02,
       -1.3102e-01,  6.0500e-01,  1.6391e+00,  2.3940e-01,  1.2128e+00,
        8.3178e-01,  7.3893e-01,  1.5200e-01, -1.4175e-01, -8.8384e-01,
        2.0829e-02, -3.2545e-01,  1.8035e+00,  1.0045e+00,  5.8484e-01,
       -6.2031e-01, -4.3296e-01,  2.3562e-01,  1.3027e+00, -8.1264e-01,
        2.3158e+00,  1.1030e+00, -6.0608e-01,  1.0101e+00, -2.2426e-01,
        1.8908e-02, -1.0931e-01,  3.8350e-01,  7.7362e-01, -8.1927e-02,
       -3.4040e-01, -1.5143e-03, -5.6640e-02,  8.7359e-01,  1.4805e+00,
        6.9421e-01, -3.0966e-01, -9.0826e-01,  3.7277e-03,  8.4550e-01])

In [11]:
def embedding_output(x):
    maxlen=10
    emb_dim=50
    embedding_out=np.zeros((x.shape[0],maxlen,emb_dim))
    
    for ix in range (x.shape[0]):
        x[ix]=x[ix].split()
        
        for ij in range(len(x[ix])):
            try:
                embedding_out[ix][ij]=embeddings_index[x[ix][ij].lower()]
            except:
                embedding_out[ix][ij]=np.zeros((50,))
                
    return embedding_out

In [12]:
embeddings_matrix_train=embedding_output(X_train)
embeddings_matrix_test=embedding_output(X_test)

C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [13]:
embeddings_matrix_test.shape

(56, 10, 50)

In [14]:
embeddings_matrix_train.shape

(132, 10, 50)

In [15]:
Y_train=to_categorical(Y_train,num_classes=5)
Y_test=to_categorical(Y_test,num_classes=5)

# Training Model
## Using RNN


In [16]:
model = Sequential()
model.add(SimpleRNN(64, input_shape=(10,50), return_sequences=True))
model.add(Dropout(0.5))
model.add(SimpleRNN(64, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 10, 64)            7360      
_________________________________________________________________
dropout (Dropout)            (None, 10, 64)            0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 325       
_________________________________________________________________
activation (Activation)      (None, 5)                 0         
Total params: 15,941
Trainable params: 15,941
Non-trainable params: 0
____________________________________________________

In [17]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

checkpoint=ModelCheckpoint(
    "best_emoji_model.h5",
    monitor="val_loss",
    verbose=True,
    save_best_only=True,
    save_weights_only=False,
    
)

earlystop=EarlyStopping(monitor='val_accuracy',patience=100)

hist=model.fit(embeddings_matrix_train,Y_train,epochs=150,batch_size=64,shuffle=True,validation_split=0.2,callbacks=[checkpoint,earlystop])


Epoch 1/150
1/2 [==============>...............] - ETA: 0s - loss: 1.8290 - accuracy: 0.2812
Epoch 00001: val_loss improved from inf to 1.54707, saving model to best_emoji_model.h5
2/2 [==============================] - 0s 151ms/step - loss: 1.9071 - accuracy: 0.2381 - val_loss: 1.5471 - val_accuracy: 0.1852
Epoch 2/150
1/2 [==============>...............] - ETA: 0s - loss: 1.8847 - accuracy: 0.1719
Epoch 00002: val_loss did not improve from 1.54707
2/2 [==============================] - 0s 21ms/step - loss: 1.8423 - accuracy: 0.2190 - val_loss: 1.5612 - val_accuracy: 0.2222
Epoch 3/150
1/2 [==============>...............] - ETA: 0s - loss: 1.8233 - accuracy: 0.2656
Epoch 00003: val_loss did not improve from 1.54707
2/2 [==============================] - 0s 19ms/step - loss: 1.7692 - accuracy: 0.2476 - val_loss: 1.5702 - val_accuracy: 0.2593
Epoch 4/150
1/2 [==============>...............] - ETA: 0s - loss: 1.7432 - accuracy: 0.2969
Epoch 00004: val_loss did not improve from 1.54707
2/

Epoch 29/150
1/2 [==============>...............] - ETA: 0s - loss: 0.6970 - accuracy: 0.7344
Epoch 00029: val_loss improved from 1.32286 to 1.29785, saving model to best_emoji_model.h5
2/2 [==============================] - 0s 67ms/step - loss: 0.7497 - accuracy: 0.7524 - val_loss: 1.2978 - val_accuracy: 0.4074
Epoch 30/150
1/2 [==============>...............] - ETA: 0s - loss: 0.6597 - accuracy: 0.8281
Epoch 00030: val_loss did not improve from 1.29785
2/2 [==============================] - 0s 18ms/step - loss: 0.6366 - accuracy: 0.8095 - val_loss: 1.2986 - val_accuracy: 0.4444
Epoch 31/150
1/2 [==============>...............] - ETA: 0s - loss: 0.6543 - accuracy: 0.7500
Epoch 00031: val_loss did not improve from 1.29785
2/2 [==============================] - 0s 16ms/step - loss: 0.6522 - accuracy: 0.7714 - val_loss: 1.3497 - val_accuracy: 0.4444
Epoch 32/150
1/2 [==============>...............] - ETA: 0s - loss: 0.5404 - accuracy: 0.8125
Epoch 00032: val_loss did not improve from 1.2

1/2 [==============>...............] - ETA: 0s - loss: 0.2450 - accuracy: 0.9375
Epoch 00058: val_loss did not improve from 1.20729
2/2 [==============================] - 0s 18ms/step - loss: 0.2402 - accuracy: 0.9238 - val_loss: 1.2284 - val_accuracy: 0.5926
Epoch 59/150
1/2 [==============>...............] - ETA: 0s - loss: 0.1856 - accuracy: 0.9531
Epoch 00059: val_loss did not improve from 1.20729
2/2 [==============================] - 0s 16ms/step - loss: 0.1833 - accuracy: 0.9619 - val_loss: 1.3193 - val_accuracy: 0.5926
Epoch 60/150
1/2 [==============>...............] - ETA: 0s - loss: 0.1295 - accuracy: 0.9688
Epoch 00060: val_loss did not improve from 1.20729
2/2 [==============================] - 0s 16ms/step - loss: 0.2021 - accuracy: 0.9333 - val_loss: 1.3620 - val_accuracy: 0.5926
Epoch 61/150
1/2 [==============>...............] - ETA: 0s - loss: 0.1804 - accuracy: 0.9688
Epoch 00061: val_loss did not improve from 1.20729
2/2 [==============================] - 0s 17ms/st

1/2 [==============>...............] - ETA: 0s - loss: 0.0378 - accuracy: 1.0000
Epoch 00088: val_loss did not improve from 1.20729
2/2 [==============================] - 0s 19ms/step - loss: 0.0509 - accuracy: 1.0000 - val_loss: 1.5655 - val_accuracy: 0.5556
Epoch 89/150
1/2 [==============>...............] - ETA: 0s - loss: 0.0378 - accuracy: 1.0000
Epoch 00089: val_loss did not improve from 1.20729
2/2 [==============================] - 0s 18ms/step - loss: 0.0470 - accuracy: 1.0000 - val_loss: 1.5198 - val_accuracy: 0.5926
Epoch 90/150
1/2 [==============>...............] - ETA: 0s - loss: 0.0566 - accuracy: 1.0000
Epoch 00090: val_loss did not improve from 1.20729
2/2 [==============================] - 0s 17ms/step - loss: 0.0552 - accuracy: 1.0000 - val_loss: 1.4655 - val_accuracy: 0.5926
Epoch 91/150
1/2 [==============>...............] - ETA: 0s - loss: 0.0491 - accuracy: 1.0000
Epoch 00091: val_loss did not improve from 1.20729
2/2 [==============================] - 0s 17ms/st

Epoch 118/150
1/2 [==============>...............] - ETA: 0s - loss: 0.0322 - accuracy: 1.0000
Epoch 00118: val_loss did not improve from 1.20729
2/2 [==============================] - 0s 17ms/step - loss: 0.0357 - accuracy: 1.0000 - val_loss: 1.4434 - val_accuracy: 0.6296
Epoch 119/150
1/2 [==============>...............] - ETA: 0s - loss: 0.0275 - accuracy: 1.0000
Epoch 00119: val_loss did not improve from 1.20729
2/2 [==============================] - 0s 17ms/step - loss: 0.0260 - accuracy: 1.0000 - val_loss: 1.4964 - val_accuracy: 0.6296
Epoch 120/150
1/2 [==============>...............] - ETA: 0s - loss: 0.0272 - accuracy: 1.0000
Epoch 00120: val_loss did not improve from 1.20729
2/2 [==============================] - 0s 16ms/step - loss: 0.0267 - accuracy: 1.0000 - val_loss: 1.5178 - val_accuracy: 0.5556
Epoch 121/150
1/2 [==============>...............] - ETA: 0s - loss: 0.0387 - accuracy: 1.0000
Epoch 00121: val_loss did not improve from 1.20729
2/2 [===========================

Epoch 148/150
1/2 [==============>...............] - ETA: 0s - loss: 0.0131 - accuracy: 1.0000
Epoch 00148: val_loss did not improve from 1.20729
2/2 [==============================] - 0s 17ms/step - loss: 0.0175 - accuracy: 1.0000 - val_loss: 1.5079 - val_accuracy: 0.6296
Epoch 149/150
1/2 [==============>...............] - ETA: 0s - loss: 0.0131 - accuracy: 1.0000
Epoch 00149: val_loss did not improve from 1.20729
2/2 [==============================] - 0s 15ms/step - loss: 0.0156 - accuracy: 1.0000 - val_loss: 1.5268 - val_accuracy: 0.6296
Epoch 150/150
1/2 [==============>...............] - ETA: 0s - loss: 0.0157 - accuracy: 1.0000
Epoch 00150: val_loss did not improve from 1.20729
2/2 [==============================] - 0s 16ms/step - loss: 0.0160 - accuracy: 1.0000 - val_loss: 1.5771 - val_accuracy: 0.6296


In [18]:
model.load_weights("best_emoji_model.h5")
model.evaluate(embeddings_matrix_test,Y_test)


2/2 [==============================] - 0s 8ms/step - loss: 1.5113 - accuracy: 0.5357


[1.5113459825515747, 0.5357142686843872]

In [19]:
pred=model.predict_classes(embeddings_matrix_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [20]:
for i in range(30):
    print(' '.join(X_test[i]),end=" ")
    print(emoji.emojize( emoji_dict[np.argmax(Y_test[i])], use_aliases=True),end=" " )
    print(emoji.emojize(emoji_dict[pred[i]], use_aliases=True))

I want to eat 🍴 😄
he did not answer 😞 😞
he got a raise 😄 ❤
she got me a present ❤ ❤
ha ha ha it was so funny 😄 😄
he is a good friend ❤ 😄
I am upset ❤ 😞
We had such a lovely dinner tonight ❤ ⚾
where is the food 🍴 🍴
Stop making this joke ha ha ha 😄 😄
where is the ball ⚾ ⚾
work is hard 😞 🍴
This girl is messing with me 😞 😄
are you serious ha ha 😄 😞
Let us go play baseball ⚾ ⚾
This stupid grader is not working 😞 😞
work is horrible 😞 😄
Congratulation for having a baby 😄 😄
stop messing around 😞 😞
any suggestions for dinner 🍴 🍴
I love taking breaks ❤ ❤
you brighten my day 😄 🍴
I boiled rice 🍴 ❤
she is a bully 😞 😄
Why are you feeling bad 😞 😞
I am upset 😞 😞
I worked during my birthday 😞 😄
My grandmother is the love of my life ❤ 😞
enjoy your break 😄 🍴
valentine day is near ❤ 🍴


In [21]:
# Predicting for our random sentence
x1 = ['I', 'love', 'dogs']
#x = "I love dogs"

x1_ = np.zeros((1,10,50))

for ix in range(len(x1)):
    x1_[0][ix] = embeddings_index[x1[ix].lower()]

x2 = ['I', 'am','happy']

x2_ = np.zeros((1,10,50))

for ix in range(len(x2)):
    x2_[0][ix] = embeddings_index[x2[ix].lower()]


In [22]:
pred=model.predict_classes(x1_)
print(' '.join(x1),end=" ")
print(emoji.emojize(emoji_dict[pred[0]], use_aliases=True),end="\n")
pred=model.predict_classes(x2_)
print(' '.join(x2),end=" ")
print(emoji.emojize(emoji_dict[pred[0]], use_aliases=True))

I love dogs ❤
I am happy 😄


# Training Model
## Using LSTM


In [23]:
model=Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 64)            29440     
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
__________________________________________________

In [24]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

checkpoint=ModelCheckpoint(
    "best_emoji_model2.h5",
    monitor="val_loss",
    verbose=True,
    save_best_only=True,
    save_weights_only=False,
    
)

earlystop=EarlyStopping(monitor='val_accuracy',patience=100)

hist=model.fit(embeddings_matrix_train,Y_train,epochs=150,batch_size=32,shuffle=True,validation_split=0.2,callbacks=[checkpoint,earlystop])


Epoch 1/150
1/4 [======>.......................] - ETA: 0s - loss: 1.5777 - accuracy: 0.2812
Epoch 00001: val_loss improved from inf to 1.61270, saving model to best_emoji_model2.h5
4/4 [==============================] - 1s 143ms/step - loss: 1.5772 - accuracy: 0.2952 - val_loss: 1.6127 - val_accuracy: 0.1852
Epoch 2/150
1/4 [======>.......................] - ETA: 0s - loss: 1.5797 - accuracy: 0.1250
Epoch 00002: val_loss did not improve from 1.61270
4/4 [==============================] - 0s 14ms/step - loss: 1.5597 - accuracy: 0.2000 - val_loss: 1.6362 - val_accuracy: 0.2222
Epoch 3/150
1/4 [======>.......................] - ETA: 0s - loss: 1.4092 - accuracy: 0.4375
Epoch 00003: val_loss did not improve from 1.61270
4/4 [==============================] - 0s 16ms/step - loss: 1.5061 - accuracy: 0.3143 - val_loss: 1.6713 - val_accuracy: 0.2222
Epoch 4/150
1/4 [======>.......................] - ETA: 0s - loss: 1.4988 - accuracy: 0.3438
Epoch 00004: val_loss did not improve from 1.61270
4

Epoch 30/150
1/4 [======>.......................] - ETA: 0s - loss: 0.1529 - accuracy: 0.9688
Epoch 00030: val_loss did not improve from 0.94836
4/4 [==============================] - 0s 12ms/step - loss: 0.3239 - accuracy: 0.8952 - val_loss: 1.2209 - val_accuracy: 0.6667
Epoch 31/150
1/4 [======>.......................] - ETA: 0s - loss: 0.2595 - accuracy: 0.9375
Epoch 00031: val_loss did not improve from 0.94836
4/4 [==============================] - 0s 14ms/step - loss: 0.3453 - accuracy: 0.8667 - val_loss: 1.2205 - val_accuracy: 0.6296
Epoch 32/150
1/4 [======>.......................] - ETA: 0s - loss: 0.3940 - accuracy: 0.8750
Epoch 00032: val_loss did not improve from 0.94836
4/4 [==============================] - 0s 15ms/step - loss: 0.2464 - accuracy: 0.9333 - val_loss: 1.3173 - val_accuracy: 0.5185
Epoch 33/150
1/4 [======>.......................] - ETA: 0s - loss: 0.2019 - accuracy: 0.9062
Epoch 00033: val_loss did not improve from 0.94836
4/4 [==============================]

1/4 [======>.......................] - ETA: 0s - loss: 0.0173 - accuracy: 1.0000
Epoch 00060: val_loss did not improve from 0.94836
4/4 [==============================] - 0s 14ms/step - loss: 0.1135 - accuracy: 0.9714 - val_loss: 1.7149 - val_accuracy: 0.5926
Epoch 61/150
1/4 [======>.......................] - ETA: 0s - loss: 0.1554 - accuracy: 0.9688
Epoch 00061: val_loss did not improve from 0.94836
4/4 [==============================] - 0s 13ms/step - loss: 0.2359 - accuracy: 0.9429 - val_loss: 1.3599 - val_accuracy: 0.6667
Epoch 62/150
1/4 [======>.......................] - ETA: 0s - loss: 0.2226 - accuracy: 0.9375
Epoch 00062: val_loss improved from 0.94836 to 0.92258, saving model to best_emoji_model2.h5
4/4 [==============================] - 0s 65ms/step - loss: 0.1097 - accuracy: 0.9714 - val_loss: 0.9226 - val_accuracy: 0.6667
Epoch 63/150
1/4 [======>.......................] - ETA: 0s - loss: 0.0431 - accuracy: 1.0000
Epoch 00063: val_loss did not improve from 0.92258
4/4 [==

Epoch 90/150
1/4 [======>.......................] - ETA: 0s - loss: 0.0041 - accuracy: 1.0000
Epoch 00090: val_loss did not improve from 0.92258
4/4 [==============================] - 0s 14ms/step - loss: 0.0066 - accuracy: 1.0000 - val_loss: 1.5751 - val_accuracy: 0.5926
Epoch 91/150
1/4 [======>.......................] - ETA: 0s - loss: 0.0061 - accuracy: 1.0000
Epoch 00091: val_loss did not improve from 0.92258
4/4 [==============================] - 0s 14ms/step - loss: 0.0071 - accuracy: 1.0000 - val_loss: 1.5660 - val_accuracy: 0.5926
Epoch 92/150
1/4 [======>.......................] - ETA: 0s - loss: 0.0112 - accuracy: 1.0000
Epoch 00092: val_loss did not improve from 0.92258
4/4 [==============================] - 0s 13ms/step - loss: 0.0077 - accuracy: 1.0000 - val_loss: 1.5551 - val_accuracy: 0.5926
Epoch 93/150
1/4 [======>.......................] - ETA: 0s - loss: 0.0057 - accuracy: 1.0000
Epoch 00093: val_loss did not improve from 0.92258
4/4 [==============================]

In [25]:
model.load_weights("best_emoji_model2.h5")
model.evaluate(embeddings_matrix_test,Y_test)


2/2 [==============================] - 0s 7ms/step - loss: 1.6048 - accuracy: 0.6429


[1.6048033237457275, 0.6428571343421936]

In [26]:
pred=model.predict_classes(embeddings_matrix_test)

In [27]:
for i in range(30):
    print(' '.join(X_test[i]),end=" ")
    print(emoji.emojize( emoji_dict[np.argmax(Y_test[i])], use_aliases=True),end=" " )
    print(emoji.emojize(emoji_dict[pred[i]], use_aliases=True))

I want to eat 🍴 🍴
he did not answer 😞 😞
he got a raise 😄 😄
she got me a present ❤ 😄
ha ha ha it was so funny 😄 😄
he is a good friend ❤ 😄
I am upset ❤ ⚾
We had such a lovely dinner tonight ❤ 😄
where is the food 🍴 🍴
Stop making this joke ha ha ha 😄 😄
where is the ball ⚾ ⚾
work is hard 😞 😄
This girl is messing with me 😞 ❤
are you serious ha ha 😄 😞
Let us go play baseball ⚾ ⚾
This stupid grader is not working 😞 😞
work is horrible 😞 😞
Congratulation for having a baby 😄 😄
stop messing around 😞 😞
any suggestions for dinner 🍴 🍴
I love taking breaks ❤ ❤
you brighten my day 😄 ❤
I boiled rice 🍴 🍴
she is a bully 😞 ❤
Why are you feeling bad 😞 😞
I am upset 😞 ⚾
I worked during my birthday 😞 😄
My grandmother is the love of my life ❤ ❤
enjoy your break 😄 ⚾
valentine day is near ❤ 😄


In [28]:
# Predicting for our random sentence
x1 = ['I', 'love', 'dogs']
#x = "I love dogs"

x1_ = np.zeros((1,10,50))

for ix in range(len(x1)):
    x1_[0][ix] = embeddings_index[x1[ix].lower()]

x2 = ['I', 'am','happy']

x2_ = np.zeros((1,10,50))

for ix in range(len(x2)):
    x2_[0][ix] = embeddings_index[x2[ix].lower()]


In [29]:
pred=model.predict_classes(x1_)
print(' '.join(x1),end=" ")
print(emoji.emojize(emoji_dict[pred[0]], use_aliases=True),end="\n")
pred=model.predict_classes(x2_)
print(' '.join(x2),end=" ")
print(emoji.emojize(emoji_dict[pred[0]], use_aliases=True))

I love dogs ❤
I am happy 😄
